In [1]:
import os

In [2]:
os.getcwd()

'c:\\projects_endtoend\\chest_cancer_detection\\notebook'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'c:\\projects_endtoend\\chest_cancer_detection'

## Update the entity

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:    
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_data : Path
    params_epochs : int
    params_batch_size : int
    params_image_size : list
    params_learning_rate : float
    params_is_augmentation : bool

## Update the configuration manager in src config

In [ ]:
from cnn_classifier.constants import *
from cnn_classifier.utils.utils import read_yaml, create_directories
import tensorflow as tf

In [ ]:
class ConfigurationManager:

    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)  ## reading config.yaml file 
        self.params = read_yaml(params_filepath)  ## reading params.yaml file

        create_directories([self.config.artifacts_root])  ## create artifacts folder

    ## model training related config

    def get_model_training_config(self) -> ModelTrainingConfig:  
        config = self.config.model_training  
        base_model = self.config.base_model  
        params = self.params                 
        training_data = os.path.join(self.config.data_ingestion.unzip_dir)

        create_directories([config.root_dir])

        training_config = ModelTrainingConfig(
            root_dir = Path(config.root_dir),
            trained_model_path = Path(config.trained_model_path),
            updated_base_model_path = Path(base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_image_size = params.IMAGE_SIZE,
            params_learning_rate = params.LEARNING_RATE,
            params_is_augmentation = params.AUGMENTATION
        )

        return training_config


    

## Update the Components

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
tf.config.run_functions_eagerly(True)
import time

In [9]:
class Training:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        self.model.compile(
        optimizer=tf.keras.optimizers.SGD(
            learning_rate=self.config.params_learning_rate
        ),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"]
    )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

## Update the Pipeline

In [10]:
try:

    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_trainer = Training(config=model_training_config)

    model_trainer.get_base_model()
    model_trainer.train_valid_generator()
    model_trainer.train()

except Exception as e:
    raise e

[2025-11-01 15:35:29,233: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2025-11-01 15:35:29,238: INFO: utils: yaml file: params.yaml loaded successfully]
[2025-11-01 15:35:29,240: INFO: utils: created directory at: artifacts]
[2025-11-01 15:35:29,242: INFO: utils: created directory at: artifacts/model_training]
[2025-11-01 15:35:29,677: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 1045 images belonging to 3 classes.
Found 4183 images belonging to 3 classes.


c:\projects_endtoend\chest_cancer_detection\cnn_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\projects_endtoend\chest_cancer_detection\cnn_env\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


261/261 ━━━━━━━━━━━━━━━━━━━━ 5524s 21s/step - accuracy: 0.4248 - loss: 15.6514 - val_accuracy: 0.4663 - val_loss: 11.8598
[2025-11-01 17:07:35,198: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
